# Comparação de Algoritmos de Classificação para Detecção de Veículos em Imagens

## Introdução

Este trabalho tem como objetivo analisar e comparar a precisão e o tempo de processamento de diferentes algoritmos de classificação aplicados à detecção de veículos em imagens. Em particular, será avaliar o desempenho do algoritmo KNN (K-Nearest Neighbors) e do algoritmo Random Forest, em comparação com uma rede neural convolucional conhecida como YOLO (You Only Look Once).

A detecção de veículos em imagens desempenha um papel fundamental em diversas aplicações, como sistemas de segurança viária, monitoramento de tráfego e veículos autônomos. Nesse contexto, é essencial avaliar e selecionar algoritmos de classificação eficientes e precisos.

## Metodologia

### Algoritmos de Classificação

1. KNN (K-Nearest Neighbors): O KNN é um algoritmo de aprendizado supervisionado que classifica objetos com base na proximidade com seus vizinhos mais próximos. Será utilizado com diferentes valores para o parâmetro K para avaliar o impacto na precisão e no tempo de classificação.

2. Random Forest: O Random Forest é um algoritmo que utiliza múltiplas árvores de decisão para realizar a classificação. Cada árvore é treinada com uma amostra aleatória dos dados e a classificação final é determinada por votação. Serão explorados diferentes números de árvores para verificar a influência na precisão e no tempo de processamento.

3. Perceptron: O Perceptron é um modelo de neurônio artificial desenvolvido na década de 1950. É considerado um dos blocos de construção fundamentais das redes neurais artificiais.

4. YOLO (You Only Look Once): O YOLO é uma rede neural convolucional especializada em detecção de objetos em tempo real. Ele é conhecido por sua eficiência e desempenho em tarefas de detecção. Será utilizado um modelo pré-treinado do YOLO para comparar a precisão e o tempo de classificação com os outros algoritmos.

### Extração de Características

Antes de aplicar os algoritmos de classificação, é necessario uma função de extração de características para representar as imagens. Para isso, foi utilizada a técnica de extração de características baseada em redes neurais convolucionais. Foi feito o uso de uma arquitetura de uma rede pré-treinada, no caso desse trabalho foi utilizado a VGG16, onde, a camada de classificação final foi removida. A saída dessa camada de extração será a representação de características da imagem. Essas características serão utilizadas como entrada para os algoritmos de classificação.


### Conjunto de Dados

Será utilizado o conjunto de dados COCO (Common Objects in Context), que é amplamente utilizado na área de visão computacional. Esse conjunto de dados contém uma variedade de objetos, incluindo veículos, com anotações precisas.

## Resultados

Nesta etapa inicial do trabalho, foi feito o download e o tratamento das imagens do conjunto de dados selecionado. Esse processo permitiu que as imagens fossem utilizadas pelos modelos disponíveis na biblioteca do scikit-learn.

Após a preparação dos dados, foi aplicado diretamente as 3000 primeiras imagens nos três modelos mencionados: KNN (K-Nearest Neighbors), Random Forest e Perceptron. Os resultados obtidos foram os seguintes:

Precisão:

*   KNN: 72%
*   Random Forest: 74%
*   Perceptron: 76%

Tempo de classificação para 20% das amostras:

* KNN: 4.4s
* Random Forest: 0.25s
* Perceptron:  0.27s


Esses resultados fornecem uma visão inicial do desempenho dos modelos em relação à classificação de veículos nas imagens. No entanto, é importante ressaltar que ainda está trabalhando com um subconjunto do dataset e que esses resultados podem variar à medida que o experimento se expande para o conjunto completo de dados.

## Conclusão
Levando em consideração apenas a métrica de precisão, os três algoritmos (KNN, Random Forest e Perceptron) demonstraram resultados promissores na tarefa de classificação de veículos em imagens.

É importante destacar que, devido à natureza bidimensional dos dados, as operações de extração de características podem ser demoradas à medida que o número de amostras aumenta. Além disso, a API do COCO para Python, que fornece um link para as imagens, soma uma latência de rede adicional ao tempo de processamento.

Embora os resultados preliminares apresentem uma precisão promissora, é essencial considerar outros fatores, como o tempo de processamento e a escalabilidade dos modelos. A medida que o tamanho do conjunto de dados aumenta, a eficiência dos algoritmos em termos de tempo de execução se torna crucial.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from pycocotools.coco import COCO
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
import tempfile
import os
import itertools
import random
import time
from sklearn.preprocessing import MultiLabelBinarizer


### Baixar e descompactar o dataset

In [ ]:
!wget -nv --show-progress wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip

wget: unable to resolve host address ‘wget’
annotations_trainva 100%[===================>] 241.19M  96.7MB/s    in 2.5s    
2023-07-10 12:08:22 URL:http://images.cocodataset.org/annotations/annotations_trainval2017.zip [252907541/252907541] -> "annotations_trainval2017.zip" [1]
FINISHED --2023-07-10 12:08:22--
Total wall clock time: 2.7s
Downloaded: 1 files, 241M in 2.5s (96.7 MB/s)


In [ ]:
!unzip -o annotations_trainval2017.zip

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


### Utilizando a lib pycocotools para filtrar e carregar as anotações do dataset

Foi filtrado em três categorias:

* car
* truck
* bus


Foi realizado também o shuffle dos dados.

In [ ]:
coco = COCO("./annotations/instances_train2017.json")

loading annotations into memory...
Done (t=21.02s)
creating index...
index created!


In [ ]:
cars_id = coco.getCatIds(catNms=['car','truck','bus'])
print(cars_id)

[3, 6, 8]


In [ ]:
ann_ids = coco.getAnnIds(catIds=cars_id)
len(ann_ids)

59909

In [ ]:
all_ann = coco.loadAnns(ann_ids)
random.shuffle(all_ann)

In [ ]:
image_urls = []
for ann in all_ann:
    image_id = ann['image_id']
    img_info = coco.loadImgs(image_id)[0]
    image_url = img_info['coco_url']
    image_urls.append(image_url)

In [ ]:
len(image_urls)

59909

In [ ]:
def load_data(max_samples, all_ann):
    vgg_model = VGG16(weights='imagenet', include_top=False)
    y = []
    df_rows = []  # Lista para armazenar as linhas do DataFrame
    num_features = None

    for i, cur_ann in enumerate(all_ann):
        if max_samples is not None and i >= max_samples:
            break

        image_info = coco.loadImgs(cur_ann["image_id"])
        image_url = image_info[0]["coco_url"]
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((224, 224))
        img = img.convert('RGB')
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        features = vgg_model.predict(img)

        if num_features is None:
            num_features = features.shape[1] * features.shape[2] * features.shape[3]
        else:
            assert num_features == features.shape[1] * features.shape[2] * features.shape[3], "Número de características inconsistente."

        cbbox = cur_ann["bbox"]
        area_obj = cur_ann["area"]
        filename = image_info[0]["file_name"]
        cur_class = coco.loadCats(cur_ann['category_id'])[0]['name']
        width, height = image_info[0]["width"], image_info[0]["height"]
        area_img = width * height

        features = features.flatten()  # Achatando as características nas dimensões das características

        df_rows.append([filename, cur_class, area_img, area_obj] + list(features))

    column_names = ["filename", "class", "area_img", "area_obj"] + [f"feature_{i+1}" for i in range(num_features)]
    df = pd.DataFrame(df_rows, columns=column_names)
    return df


In [ ]:
df = load_data(3000,all_ann)

1/1 [==============================] - 1s 561ms/step


In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
df.shape

(3000, 25092)

In [ ]:
df.head(5)

,filename,class,area_img,area_obj,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_25079,feature_25080,feature_25081,feature_25082,feature_25083,feature_25084,feature_25085,feature_25086,feature_25087,feature_25088
0,000000385346.jpg,bus,273920,23622.27510,0.0,0.0,0.0,0.0,15.876854,0.0,...,7.886225,0.0,0.0,23.913578,0.000000,0.0,12.927351,0.0,6.375592,0.000000
1,000000222587.jpg,car,273280,3890.63625,0.0,0.0,0.0,0.0,58.089596,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2,000000271580.jpg,car,307200,482.84500,0.0,0.0,0.0,0.0,0.000000,0.0,...,5.905355,0.0,0.0,0.000000,9.000976,0.0,0.000000,0.0,0.000000,0.000000
3,000000187181.jpg,car,307200,187.44185,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,9.855352,10.935303
4,000000460342.jpg,bus,307200,30768.99365,0.0,0.0,0.0,0.0,4.812307,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,5.827429


In [ ]:
y = df['class']
X = df.drop(['filename','class'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [ ]:
knn_model = KNeighborsClassifier()

knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
inicio = time.time()
y_pred_knn = knn_model.predict(X_test)
fim = time.time()
accuracy_knn = accuracy_score(y_test, y_pred_knn)
time_knn = fim - inicio
print("KNN Accuracy:", accuracy_knn)
print("KNN Time:", time_knn)

KNN Accuracy: 0.72
KNN Time: 4.399287700653076


In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
inicio = time.time()
y_pred_rf = rf_model.predict(X_test)
fim = time.time()
accuracy_rf = accuracy_score(y_test, y_pred_rf)
time_rf = fim - inicio
print("RandomForest Accuracy:", accuracy_rf)
print("RandomForest Time:", time_rf)

RandomForest Accuracy: 0.7416666666666667
RandomForest Time: 0.2501397132873535


In [ ]:
perceptron_model = Perceptron()
perceptron_model.fit(X_train, y_train)

Perceptron()

In [ ]:
inicio = time.time()
y_pred_perceptron = perceptron_model.predict(X_test)
fim = time.time()
accuracy_perceptron = accuracy_score(y_test, y_pred_perceptron)
time_perceptron = fim - inicio
print("Perceptron Accuracy:", accuracy_perceptron)
print("Perceptron Time:", time_perceptron)

Perceptron Accuracy: 0.76
Perceptron Time: 0.27925825119018555
